This code was from components page on landlab

In [ ]:

import numpy as np
from landlab import RasterModelGrid
from landlab.components import (FlowAccumulator, DepressionFinderAndRouter, Space, FastscapeEroder)
np.random.seed(seed = 5000)

In [ ]:
mg = RasterModelGrid((5,5), xy_spacing=10.0)
_ = mg.add_zeros('topographic__elevation', at='node')
mg.at_node['topographic__elevation'] += (mg.node_y / 10. + mg.node_x / 10. + np.random.rand(len(mg.node_y)) / 10.)

mg.set_closed_boundaries_at_grid_edges(bottom_is_closed=True, left_is_closed=True,right_is_closed=True,top_is_closed=True)
mg.set_watershed_boundary_condition_outlet_id(0, mg.at_node['topographic__elevation'], -9999.)
fsc_dt = 100.
space_dt = 100.




In [ ]:
fr = FlowAccumulator(mg, flow_director='D8')
df = DepressionFinderAndRouter(mg)
fsc = FastscapeEroder(mg, K_sp = .001,m_sp=.5, n_sp=1)


In [ ]:
for x in range(100):
    fr.run_one_step()
    df.map_depressions()
    fsc.run_one_step(dt=fsc_dt)
    mg.at_node['topographic__elevation'][0] -= 0.001 # Uplift


In [ ]:
_ = mg.add_zeros('soil__depth', at='node', dtype=float)
mg.at_node['soil__depth'] += 0.5
mg.at_node['topographic__elevation'] += mg.at_node['soil__depth']


In [ ]:
ha = Space(mg, K_sed=0.00001,K_br=0.00000000001,F_f=0.5, phi=0.1, H_star=1., v_s=0.001, m_sp=0.5,  n_sp = 1.0, sp_crit_sed=0, sp_crit_br=0)


In [ ]:
for x in range(2000): #Space component loop
    fr.run_one_step()
    df.map_depressions()
    ha.run_one_step(dt=space_dt)
    mg.at_node['bedrock__elevation'][0] -= 2e-6 * space_dt

In [ ]:
np.around(mg.at_node['soil__depth'], decimals=3) # doctest: +NORMALIZE_WHITESPACE


In [ ]:
np.around(mg.at_node['topographic__elevation'], decimals=3) 

In [ ]:
_name = "Space"
_unit_agnostic = True

_info = {
    "flow__link_to_receiver_node": {
        "dtype": int,
        "intent": "in",
        "optional": False,
        "units": "-",
        "mapping": "node",
        "doc": "ID of link downstream of each node, which carries the discharge",
    },
    "flow__receiver_node": {
        "dtype": int,
        "intent": "in",
        "optional": False,
        "units": "-",
        "mapping": "node",
        "doc": "Node array of receivers (node that receives flow from current node)",
    },
    "flow__upstream_node_order": {
        "dtype": int,
        "intent": "in",
        "optional": False,
        "units": "-",
        "mapping": "node",
        "doc": "Node array containing downstream-to-upstream ordered list of node IDs",
    },
    "sediment__influx": {
        "dtype": float,
        "intent": "out",
        "optional": False,
        "units": "m3/s",
        "mapping": "node",
        "doc": "Sediment flux (volume per unit time of sediment entering each node)",
    },
    "sediment__outflux": {
        "dtype": float,
        "intent": "out",
        "optional": False,
        "units": "m3/s",
        "mapping": "node",
        "doc": "Sediment flux (volume per unit time of sediment leaving each node)",
    },
    "soil__depth": {
        "dtype": float,
        "intent": "inout",
        "optional": False,
        "units": "m",
        "mapping": "node",
        "doc": "Depth of soil or weathered bedrock",
    },
    "surface_water__discharge": {
        "dtype": float,
        "intent": "in",
        "optional": False,
        "units": "m**3/s",
        "mapping": "node",
        "doc": "Volumetric discharge of surface water",
    },
    "topographic__elevation": {
        "dtype": float,
        "intent": "inout",
        "optional": False,
        "units": "m",
        "mapping": "node",
        "doc": "Land surface topographic elevation",
    },
    "topographic__steepest_slope": {
        "dtype": float,
        "intent": "in",
        "optional": False,
        "units": "-",
        "mapping": "node",
        "doc": "The steepest *downhill* slope",
    },
}


In [ ]:
def __init__(
    self,
    grid,
    K_sed=0.002,
    K_br=0.002,
    F_f=0.0,
    phi=0.3,
    H_star=0.1,
    v_s=1.0,
    m_sp=0.5,
    n_sp=1.0,
    sp_crit_sed=0.0,
    sp_crit_br=0.0,
    discharge_field="surface_water__discharge",
    solver="basic",
    dt_min=DEFAULT_MINIMUM_TIME_STEP,):